# Fiscal Intelligence & SDG Alignment Analysis

**Objective**: Transform 10Alytics fiscal dataset into actionable insights for policymakers and business leaders.

**Data Source**: `10Alytics Hackathon- Fiscal Data.xlsx` (cleaned via `scripts/fiscal_data_audit.py`)

**Audience**: Government finance ministries, business strategy teams, development institutions.

**Scope**: 14 African countries, 25 indicators (1960–2025), SDG-mapped recommendations.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')
sns.set_theme(style='whitegrid', palette='muted')
plt.rcParams['figure.figsize'] = (14, 7)
plt.rcParams['font.size'] = 11

%matplotlib inline


## 1. Data Loading & Exploration


In [ ]:
# Load cleaned fiscal dataset
data_path = Path('../data/processed/fiscal_data_clean.csv')
df = pd.read_csv(data_path, parse_dates=['Time', 'Time_aligned'])

# Load stress scorecard
scorecard_path = Path('../data/processed/fiscal_stress_scorecard.csv')
scorecard = pd.read_csv(scorecard_path)

print(f"Loaded {len(df):,} fiscal records")
print(f"Countries: {df['Country'].nunique()}")
print(f"Indicators: {df['Indicator'].nunique()}")
print(f"Time range: {df['Year'].min():.0f}–{df['Year'].max():.0f}")

df.head()


## 2. Fiscal Stress Scorecard Analysis


In [ ]:
# Focus on top-5 economies by data richness
focus_countries = ['Nigeria', 'Ghana', 'Kenya', 'South Africa', 'Egypt']
scorecard_focus = scorecard[scorecard['Country'].isin(focus_countries)].copy()

print("Latest fiscal stress indicators (top-5 economies):\n")
display_cols = [
    'Country', 'Debt_to_GDP', 'Deficit_to_Revenue', 'Revenue_to_GDP',
    'Inflation Rate', 'Trade_Balance_to_GDP'
]
scorecard_focus[display_cols]


In [ ]:
# Visualise debt-to-GDP vs. revenue-to-GDP
fig, ax = plt.subplots(figsize=(12, 7))

for _, row in scorecard_focus.iterrows():
    debt_gdp = row['Debt_to_GDP']
    rev_gdp = row['Revenue_to_GDP']
    if pd.notna(debt_gdp) and pd.notna(rev_gdp):
        ax.scatter(rev_gdp*100, debt_gdp*100, s=200, alpha=0.7)
        ax.text(rev_gdp*100, debt_gdp*100, f"  {row['Country']}", fontsize=11, va='center')

ax.axhline(90, color='red', linestyle='--', linewidth=1.5, alpha=0.6, label='Debt/GDP 90% threshold')
ax.axvline(25, color='green', linestyle='--', linewidth=1.5, alpha=0.6, label='Revenue/GDP 25% target')

ax.set_xlabel('Revenue to GDP (%)', fontsize=13, weight='bold')
ax.set_ylabel('Debt to GDP (%)', fontsize=13, weight='bold')
ax.set_title('Fiscal Stress Map: Debt Burden vs. Revenue Mobilisation', fontsize=15, weight='bold', pad=20)
ax.legend(loc='best', fontsize=10)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()


## 3. SDG-Mapped Policy Recommendations


In [ ]:
# Build policy recommendation matrix
recommendations = []

for _, row in scorecard_focus.iterrows():
    country = row['Country']
    debt_gdp = row['Debt_to_GDP']
    deficit_rev = row['Deficit_to_Revenue']
    rev_gdp = row['Revenue_to_GDP']
    inflation = row['Inflation Rate']
    trade_bal = row['Trade_Balance_to_GDP']
    
    issues = []
    sdgs = []
    actions = []
    biz_impact = []
    
    if pd.notna(debt_gdp) and debt_gdp > 0.9:
        issues.append('High debt burden')
        sdgs.extend(['SDG 9', 'SDG 16'])
        actions.append('Debt reprofiling + PPP infrastructure financing')
        biz_impact.append('Reassess sovereign exposure; pursue blended-finance opportunities')
    
    if pd.notna(deficit_rev) and deficit_rev > 0.6:
        issues.append('Deficit overshoot')
        sdgs.extend(['SDG 8', 'SDG 16'])
        actions.append('Quarterly expenditure reviews + VAT compliance drive')
        biz_impact.append('Anticipate tighter procurement; hedge FX exposure')
    
    if pd.notna(rev_gdp) and rev_gdp < 0.18:
        issues.append('Weak revenue mobilisation')
        sdgs.extend(['SDG 16', 'SDG 17'])
        actions.append('Broaden tax base (digital economy, property, informal sector)')
        biz_impact.append('Prepare for widened tax net; expand compliance systems')
    
    if pd.notna(inflation) and inflation > 10:
        issues.append('Elevated inflation')
        sdgs.extend(['SDG 1', 'SDG 2'])
        actions.append('Food security buffers + calibrated interest rate path')
        biz_impact.append('Build working-capital buffers; local sourcing where feasible')
    
    if pd.notna(trade_bal) and trade_bal < -0.05:
        issues.append('Trade deficit pressure')
        sdgs.extend(['SDG 8', 'SDG 9'])
        actions.append('Export diversification + FX reserves management')
        biz_impact.append('Hedge FX volatility; pursue import-substitution opportunities')
    
    if issues:
        recommendations.append({
            'Country': country,
            'Stress Signals': '; '.join(issues),
            'SDG Linkage': ', '.join(sorted(set(sdgs))),
            'Policy Actions': '; '.join(actions),
            'Business Implications': '; '.join(biz_impact)
        })

rec_df = pd.DataFrame(recommendations)
print("Policy Recommendation Matrix:\n")
rec_df


## 4. Export Key Outputs


In [ ]:
# Export recommendation matrix
output_dir = Path('../data/processed')
rec_df.to_csv(output_dir / 'policy_recommendations.csv', index=False)
print(f"✓ Exported recommendations to {output_dir / 'policy_recommendations.csv'}")

# Export focus country time series
focus_series = df[df['Country'].isin(focus_countries)].copy()
focus_series.to_csv(output_dir / 'focus_countries_timeseries.csv', index=False)
print(f"✓ Exported focus time series to {output_dir / 'focus_countries_timeseries.csv'}")
